# Building Makemore From Scratch


**Makemore** is an educational language model by [Andrej Karpathy](https://github.com/karpathy) that demonstrates how to build text generation models from scratch.<br>
As we can see the name comes from its purpose i.e. training models that can *"make more"* examples of data (e.g., generate new names, words, or text) based on a given dataset. It shows how character-level language models are built step by step



## Part-1: Bigrams

### 1.  Loading and Inspecting The Data

In [4]:
words = open('names.txt', 'r').read().splitlines()#reading names from file

In [5]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [6]:
len(words) # total number of words

32033

In [9]:
# finding minimum and maximum length of words
min_len = min(len(w) for w in words)
max_len = max(len(w) for w in words)
print(f"Minimum length: {min_len} and Maximum length: {max_len}")

Minimum length: 2 and Maximum length: 15


### 2. Building the Bigram Model (Dictionary Approach)